# Linear Regression Tutorial

In this tutorial, we will learn how to do linear regression with scikit in Python. We will explore how to predict home prices using a variety of features, both numerical and categorical. Along the way, we will also learn about some other concepts such as correlation, mean squared error and R^2

# Goals

**(1) Data exploration/preparation: Load and understand data**

**(2) Linear Regression**
       - with a single feature
       - with multiple numerical features
       - with numerical and categorical features 

**(3) Understanding goodness of fit metrics**

**(4) Standardization**

**(5) Caveats and more things you can explore on your own**

# Data exploration: Load and understand data

In [ ]:
# Let's import necessary packages
import pandas as pd # data handling/manipulation package
import os
import numpy as np # mathematical and array operations
import matplotlib.pyplot as plt # plotting
import matplotlib.image as mpimg # plotting
import statsmodels.api as sm # statistics
import warnings # handle warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('king_county_home_prices.csv', sep = ",")

In [ ]:
image = mpimg.imread("HousingDatasetFeatures.PNG")
plt.figure(figsize = (10,10))
plt.imshow(image)
plt.show()

**Split data into train and test**

Intution for splitting data into train and test:

When you want to measure how well you have learnt something, you want to do some practice problems or take a practice exam. That is exactly what is happening here. You are putting some questions away as practice questions for yourself and learning from the rest.

In [ ]:
np.random.seed(2019)
msk = np.random.rand(len(df)) < 0.8
train = df[msk]
test = df[~msk]

**Let's look at the data**

In [ ]:
train.head()

In [ ]:
train['price'].describe()

In [ ]:
train['sqft_living'].describe()

In [ ]:
plt.scatter(train['sqft_living'], train['price'])
# titles

# Data preparation

In [ ]:
# Python package sklearn requires data to be in matrix format ie.e two dimesional array
# so we will transform single feature using the reshape() function

data_train = np.array(train['sqft_living']).reshape(-1, 1)
target_train = np.array(train['price']).reshape(-1, 1)

data_test = np.array(test['sqft_living']).reshape(-1, 1)
target_test = np.array(test['price']).reshape(-1, 1)

# Linear Regression with single feature

In linear regression, we want to fit a linear equation to our data. In the case where we have just one independent variable, we are trying to fit a line. 

The canonical equation of a line is Y = mX + c. So, two real values completely determine a line. The sole purpose of the training process is to figure out the best possible 'm'  and  'c'. The value for 'c' is non zero only if the line does not pass through the origin.

In our problem, we are solving price ~ m * sqft_living + c

In [ ]:
from sklearn.linear_model import LinearRegression
model1=LinearRegression()
model1.fit(data_train,target_train)

In [ ]:
preds = model1.predict(data_test)

In [ ]:
# Plot outputs
plt.scatter(data_test, target_test,  color='black')
plt.plot(data_test, preds, color='blue', linewidth=3)
plt.title('price ~ m * sqft_living + c')
plt.xlabel('sqft_living')
plt.ylabel('price')

In [ ]:
print('Coefficient of sqft_living or \'c\': ', model1.coef_)

This coefficient tells us that for a unit increase in sqft_living, the price of a home increases by $279.62829594

# Goodness of fit

Now that we have fit a line, we want to know how good our line is i.e. we need some way to measure how well its predictions
actually match the observed data. Below are a few metrics that are used to evaluate goodness of fit.

(1) **Mean Square error/ squared loss**

$$MSE = \frac{1}{N}\sum_{i=1}^{N} (y_i - f(x_i))^2$$

This is the most commonly used metric. However, the error does not have the same 'units' as the dependent variable i.e. price in our problem.

(2) **Root Mean Squared Error**

$$RMSE = \sqrt{\frac{1}{N}\sum_{i=1}^{N} (y_i - f(x_i))^2}$$

(3) **R^2**

This metric indicates the percentage of the variance in the value that we are trying to predict that the features explain collectively. 

$$ R^2 = \frac{\text{Variance explained by the model}}{\text{Total variance}}$$


**Understanding R^2**

In [ ]:
# Predicting using the average price gives us a notion of inherent variance in the data before regression was performed

avg_price = np.mean(target_test)
print("average price: ",avg_price)
preds_avg_price = np.array([avg_price] * len(target_test)).reshape(-1, 1)

# TSS : total sum of squares - total variance
# RSS: residual sum of squares - variance UNEXPLAINED by the model

TSS = np.sum((target_test- preds_avg_price)**2)
RSS = np.sum((target_test-preds)**2)
R_squared = (TSS-RSS)/TSS
print("total variance: ", TSS)
print("variance unexplained by model: ", RSS)
print("variance explained by model: %.2f" % R_squared)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
print("Mean Squared Error: %.2f" % mean_squared_error(target_test, preds))
print("Root Mean Squared Error: %.2f" % np.sqrt(mean_squared_error(target_test, preds)))
print('R^2: %.2f' % r2_score(target_test, preds))

# Linear Regression with multiple numerical features

In [ ]:
train.columns
num_cols = ['bedrooms', 'bathrooms', 'sqft_living' , 'floors', 'sqft_above', 'sqft_basement']

In [ ]:
df_sub = train[num_cols]

In [ ]:
fig_size = plt.rcParams["figure.figsize"] 
fig_size[0]=16.0
fig_size[1]=8.0
df_sub.hist(bins=100)
plt.show()

It looks like there are a lot of homes without a basement, so let's transform that into a categorical variable. Let's deal with that later

In [ ]:
num_cols = ['bedrooms', 'bathrooms', 'sqft_living']

#data prep
data_train = np.array(train[num_cols])
target_train = np.array(train['price'])

data_test = np.array(test[num_cols])
target_test = np.array(test['price'])

# train the model
model1=LinearRegression()
model1.fit(data_train,target_train)

# get predictions
preds = model1.predict(data_test)

# goodness of fit
print("Mean Squared Error: %.2f" % mean_squared_error(target_test, preds))
print("Root Mean Squared Error: %.2f" % np.sqrt(mean_squared_error(target_test, preds)))
print('R^2: %.2f' % r2_score(target_test, preds))
pd.DataFrame({'name':num_cols,'value':model1.coef_}).sort_values('value', ascending = False)

In [ ]:
num_cols = ['bedrooms', 'bathrooms', 'sqft_living' , 'floors', 'sqft_above', 'sqft_basement']

data_train = np.array(train[num_cols])
target_train = np.array(train['price'])

data_test = np.array(test[num_cols])
target_test = np.array(test['price'])

# train the model
model1=LinearRegression()
model1.fit(data_train,target_train)

# get predictions
preds = model1.predict(data_test)

# goodness of fit
print("Mean Squared Error: %.2f" % mean_squared_error(target_test, preds))
print("Root Mean Squared Error: %.2f" % np.sqrt(mean_squared_error(target_test, preds)))
print('R^2: %.2f' % r2_score(target_test, preds))
pd.DataFrame({'name':num_cols,'value':model1.coef_}).sort_values('value', ascending = False)

In [ ]:
num_cols = ['bedrooms', 'sqft_living', 'view']

data_train = np.array(train[num_cols])
target_train = np.array(train['price'])

data_test = np.array(test[num_cols])
target_test = np.array(test['price'])

# train the model
model1=LinearRegression()
model1.fit(data_train,target_train)

# get predictions
preds = model1.predict(data_test)

# goodness of fit
print("Mean Squared Error: %.2f" % mean_squared_error(target_test, preds))
print("Root Mean Squared Error: %.2f" % np.sqrt(mean_squared_error(target_test, preds)))
print('R^2: %.2f' % r2_score(target_test, preds))
pd.DataFrame({'name':num_cols,'value':model1.coef_}).sort_values('value', ascending = False)

By looking at the coefficients, we learn that:

- per unit increase in floors, the price increases by \$14k
- per unit increase in number of times the home was viewed, the price increases by ~\$100k
- however, notice that the number of bedrooms has a negative coefficient. This is clearly due to correlated features. After accounting for the variation explained by the other variables, the relationship between bedrooms and price is negative. Doing a multiple regression with predictors that are this highly correlated is likely to lead to flawed inferences.

In data such as this, the weights are hard to interpret because of highly correlated features. The assumption that a unit increase in one keeping others constant is impossible to hold. How do you increase the number of floors by keeping sqft_living constant?

But, we cannot order the features by their coefficients and infer feature importance because we have not standardized the data. 

Standardization is relevant when predictors i.e. independent variables are expressed in different units. Discerning importance of predictors based on the unstandardized coefficient would not be fair. This is because the units of these variable are not the same.

- Unstandardized coefficients: It represents the amount by which dependent variable changes if we change independent variable by one unit keeping other independent variables constant.

- Standardized coefficients: The standardized coefficient is measured in units of standard deviation. A beta value of 1.25 indicates that a change of one standard deviation in the independent variable results in a 1.25 standard deviations increase in the dependent variable.

# Standardization

In [ ]:
import sklearn
scaler = sklearn.preprocessing.StandardScaler()

In [ ]:
from sklearn import preprocessing

X_scaler = preprocessing.StandardScaler()
data_train_std = X_scaler.fit_transform(data_train)
data_test_std = X_scaler.transform(data_test)

y_scaler = preprocessing.StandardScaler()
target_train_std = y_scaler.fit_transform(target_train[:, None])[:, 0]
target_test_std = y_scaler.transform(target_test[:, None])[:, 0]

In [ ]:
model1=LinearRegression()
model1.fit(data_train_std,target_train_std)

In [ ]:
preds = model1.predict(data_test_std)

In [ ]:
print("Mean Squared Error: %.2f" % mean_squared_error(target_test_std, preds))
print("Root Mean Squared Error: %.2f" % np.sqrt(mean_squared_error(target_test_std, preds)))
print('R^2: %.2f' % r2_score(target_test_std, preds))

In [ ]:
pd.DataFrame({'name':num_cols,'value':model1.coef_}).sort_values('value', ascending = False)

# Feature engineering of categorical features

In [ ]:
print("Number of unique values in zipcode: ", train['zipcode'].nunique())
print("Number of unique values in yr_built: ", train['yr_built'].nunique())
print("Number of unique values in yr_renovated: ",train['yr_renovated'].nunique())
print("Number of unique values in waterfront: ",train['waterfront'].nunique())

We are going to pick a few categorical variables and see how that helps us improve our predictions. I will leave the exploration of the categorical variables to the reader. We must always keep in mind that the total number of features must not exceed the number of samples.

In [ ]:
# Let's bring back our basement feature
def to_categorical(row):
    if row > 0:
        return 1.
    else:
        return 0
    
df['sqft_basement_cat'] = df['sqft_basement'].apply(to_categorical)

In [ ]:
df.groupby('sqft_basement_cat')['id'].count()

Let's look at the distribution of homes across zipcodes. We want to make sure that the homes are well spread across zipcodes to avoid overfitting

In [ ]:
df.groupby(['zipcode'])['id'].nunique().describe()

In [ ]:
numeric_features = ['bedrooms', 'sqft_living', 'view']
categorical_features = ['waterfront', 'sqft_basement_cat', 'zipcode']

**One Hot Encoding**

Recap: One hot encoding is a representation of categorical variables in a way that is conducive to learning by algorithms. In one hot encoding, a categorical variable is reprsented as binary vectors.

In [ ]:
cat_features_list = []
for column in categorical_features:
    dfDummies = pd.get_dummies(df[column], prefix = column)
    cat_features_list = cat_features_list + dfDummies.columns.tolist()
    df = pd.concat([df, dfDummies], axis=1)
    del df[column]

In [ ]:
train = df[msk]
test = df[~msk]

In [ ]:
all_features = numeric_features+cat_features_list

In [ ]:
train[all_features].head()

# Linear Regression with numerical and categorical features

In [ ]:
data_train = np.array(train[all_features])
target_train = np.array(train['price'])

data_test = np.array(test[all_features])
target_test = np.array(test['price'])

In [ ]:
model1=LinearRegression()
model1.fit(data_train,target_train)

In [ ]:
preds = model1.predict(data_test)

In [ ]:
print("Mean Squared Error: %.2f" % mean_squared_error(target_test, preds))
print("Root Mean Squared Error: %.2f" % np.sqrt(mean_squared_error(target_test, preds)))
print('R^2: %.2f' % r2_score(target_test, preds))

# Caveats and what you can explore on your own

Things to explore on your own:
- interaction variables
- regularization
- other algorithms like decision trees, random forests
- different categorical varaibles
- understand bias variance trade off
- cross validation

Caveats:
- Highly correlated variables
- Feature importance
- exploding features

# Summary

First, this is no way a cook book that you can follow. This tutorial was only to give a basic overview of linear regression and how to implement that using standard libraries. We hope that this has piqued your interest to explore the capabilities of machine learning